In [1]:
# import library
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import random

import pymoo
from pymoo.core.problem import Problem

import joblib


import pandas as pd
import random
import math
import os

from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.optimize import minimize
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting


import csv
import os



In [2]:
# import ML model

import time
# date = time.strftime("%y%m%d")
date = 251017
import os

print("start")

model_names = [
    f"Lmt_{date}",
    f"Llt_{date}",
    f"copperloss_Tx_{date}",
    f"copperloss_Rx1_{date}",
    f"copperloss_Rx2_{date}",
    f"coreloss_{date}",
    f"magnetizing_copperloss_Tx_{date}",
    f"magnetizing_copperloss_Rx1_{date}",
    f"magnetizing_copperloss_Rx2_{date}",
    f"B_left_{date}",
    f"B_center_{date}",
    f"B_top_left_{date}"
]

model_labels = [
    "Lmt",
    "Llt", 
    "copperloss_Tx",
    "copperloss_Rx1",
    "copperloss_Rx2",
    "coreloss",
    "magnetizing_copperloss_Tx",
    "magnetizing_copperloss_Rx1",
    "magnetizing_copperloss_Rx2",
    "B_left",
    "B_center",
    "B_top_left"
]

# dict에 담아서 관리
models = {}
for name, label in zip(model_names, model_labels):
    model_path = f"model/{name}.pkl"
    
    # 파일 존재 여부 확인
    if not os.path.exists(model_path):
        print(f"모델 파일이 존재하지 않습니다: {model_path}")
        continue
        
    try:
        print(f"Loading model: {name}")
        models[label] = joblib.load(model_path)
    except Exception as e:
        print(f"모델 로딩 중 에러 발생: {name}")
        print(f"에러 메시지: {str(e)}")

start
Loading model: Lmt_251017
Loading model: Llt_251017
Loading model: copperloss_Tx_251017
Loading model: copperloss_Rx1_251017
Loading model: copperloss_Rx2_251017
Loading model: coreloss_251017
Loading model: magnetizing_copperloss_Tx_251017
Loading model: magnetizing_copperloss_Rx1_251017
Loading model: magnetizing_copperloss_Rx2_251017
Loading model: B_left_251017
Loading model: B_center_251017
Loading model: B_top_left_251017


In [3]:
import numpy as np, random, math

SCALE  = np.array([5, 50, 1, 0.1, 0.1, 0.1,
                   0.01, 1, 1, 0.1, 0.1 ,0.01, 0.1, 0.035,
                   1, 0.035, 0.1, 0.1, 0.01, 0.01, 0.1, 0.1] 
                   , dtype=float)
OFFSET = np.array([0, 100, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0],    dtype=float)

def decode_vars(X_raw: np.ndarray) -> np.ndarray:
    """
    정수형 설계변수 → 실제 물리 단위로 변환.
    X_raw : shape (n, 14) 또는 (14,)
    """
    X = np.atleast_2d(X_raw).astype(float)
    return X * SCALE + OFFSET 

def pre_processing_data_copper(X_raw) :
    # "w1","g1","g2","l1_leg","l1_top","l2","h1","w1_ratio","l1_center","ratio","pri_turns","width_ratio","pri_width","pri_height","pri_space_x","pri_x","pri_preg","sec_width","sec_x","sec_space_y",
    #                                "sec_y","sec_preg","sec_height","sec_space_x","pri_space_y","pri_y"
    X = decode_vars(X_raw)
    (freq,input_voltage,w1,l1_leg,l1_top,l2,h1,l1_center,Tx_turns,Tx_space_x,
     Tx_space_y,Tx_preg,Tx_width,Tx_height,Rx_width,Rx_height,Rx_space_x,Rx_space_y,Rx_preg,
     g2,Tx_layer_space_x,Tx_layer_space_y) = X.T
    

    new_data = np.column_stack((freq, w1, l1_leg, l1_top, l2, h1, l1_center, Tx_turns, Tx_width, Tx_height, 
                                Tx_space_x, Tx_space_y, Tx_preg, Rx_width, Rx_height, Rx_space_x, Rx_space_y,
                                Rx_preg, g2, Tx_layer_space_x, Tx_layer_space_y))
    
    return new_data

def pre_processing_data_core(X_raw,Tx_current,magnetizing_current) :
    # "w1","g1","g2","l1_leg","l1_top","l2","h1","w1_ratio","l1_center","ratio","pri_turns","width_ratio","pri_width","pri_height","pri_space_x","pri_x","pri_preg","sec_width","sec_x","sec_space_y",
    #                                "sec_y","sec_preg","sec_height","sec_space_x","pri_space_y","pri_y"
    X = decode_vars(X_raw)
    (freq,input_voltage,w1,l1_leg,l1_top,l2,h1,l1_center,Tx_turns,Tx_space_x,
     Tx_space_y,Tx_preg,Tx_width,Tx_height,Rx_width,Rx_height,Rx_space_x,Rx_space_y,Rx_preg,
     g2,Tx_layer_space_x,Tx_layer_space_y) = X.T
    

    new_data = np.column_stack((freq, w1, l1_leg, l1_top, l2, h1, l1_center, Tx_turns, Tx_width, Tx_height, 
                                Tx_space_x, Tx_space_y, Tx_preg, Rx_width, Rx_height, Rx_space_x, Rx_space_y,
                                Rx_preg, g2, Tx_layer_space_x, Tx_layer_space_y,Tx_current,magnetizing_current))
    
    
    return new_data

def compute_area(X_raw):
    X = decode_vars(X_raw)
    # ── 0. 입력 분해 (2D 배열 X) ────────────────────────────
    (freq,input_voltage,w1,l1_leg,l1_top,l2,h1,l1_center,Tx_turns,Tx_space_x,
     Tx_space_y,Tx_preg,Tx_width,Tx_height,Rx_width,Rx_height,Rx_space_x,Rx_space_y,Rx_preg,
     g2,Tx_layer_space_x,Tx_layer_space_y) = X.T

    # volume
    Area = (l1_leg + l2 + l1_center/2)*2 * (w1+Tx_layer_space_x*4+Tx_width*5+Tx_space_x+Rx_space_x+Rx_width)
    height = l1_top*2 + h1
    Volume = Area * height
    
    Tx = Tx_width * 5 + Tx_layer_space_y * 4 + Tx_space_y
    Rx = Rx_width + Rx_space_y
    max_length = l2 - (Tx+Rx)
    gap = h1 - g2

    coil_height = (Tx_height*2 + Tx_preg*2 + Rx_height*2 + Rx_preg*2)
    hh = h1  - coil_height       
    Tx_Rx = Tx_preg/2 + Rx_preg
    


    return Area, height, max_length, Tx_Rx, hh, gap, Volume


In [4]:
# "w1","l1_leg","l1_top","l2","h1","l1_center","Tx_turns","Tx_width","Tx_height","Tx_space_x","Tx_space_y",
# "Tx_preg","Rx_width","Rx_height","Rx_space_x","Rx_space_y","Rx_preg",
# "g2","Tx_layer_space_x","Tx_layer_space_y","wire_diameter","strand_number"

vars1 = { # under, upper, resolution
    "freq_range" : [1,80,1,0], #*5
    "input_voltage_range" : [1,8,1,0], #*50 + 100

    "w1_range": [20, 200, 1, 0],
    "l1_leg_range": [20, 150, 1, 0], # *0.1
    "l1_top_range": [5, 20, 1, 0], # *0.1
    "l2_range": [50, 300, 1, 0], # *0.1

    "h1_range": [10, 300, 1, 0], # *0.01 
    "l1_center_range": [1, 25, 1, 0],

    "Tx_turns_range" : [2,20,1,0],
    "Tx_space_x_range": [1, 50, 1, 0],# *0.1 
    "Tx_space_y_range": [1, 50, 1, 0],# *0.1
    "Tx_preg_range": [5, 30, 1, 0],# *0.01
    "Tx_width_range": [5, 30, 1, 0],# *0.1
    "Tx_height_range": [1, 5, 1, 0],# *0.035

    "Rx_width_range": [4, 20, 1, 0], 
    "Rx_height_range": [1, 5, 1, 0],# *0.035 
    "Rx_space_x_range": [1, 50, 1, 0],# *0.1
    "Rx_space_y_range": [1, 50, 1, 0],# *0.1
    "Rx_preg_range": [5, 30, 1, 0],# *0.01


    "g2_range" : [10,300,1,0], #*0.01

    "Tx_layer_space_x_range": [2, 50, 1, 0],# *0.1
    "Tx_layer_space_y_range": [2, 50, 1, 0]# *0.1


}

class ProblemVariables:
    def __init__(self, var_dict):
        self.var_dict = var_dict
        self.names = list(var_dict.keys())

    def get_num_of_variables(self):
        return len(self.var_dict)

    def get_lower_bounds(self):
        return np.array([v[0] for v in self.var_dict.values()])

    def get_upper_bounds(self):
        return np.array([v[1] for v in self.var_dict.values()])

class MyProblem(Problem) :

    def __init__(self, problem_vars):
        self.vars = problem_vars
        super().__init__(n_var=self.vars.get_num_of_variables(),     #number of inputs
                    n_obj=2,     #number of outputs
                    n_constr=11,  #nubmer of constraints
                    xl=np.array(self.vars.get_lower_bounds()), #input lower bounds
                    xu=np.array(self.vars.get_upper_bounds()), #input upper bounds
                    type_var = int)
 
    #[N1, N2, freq, per, w1, l1, l2, h1, l1_leg, l1_top, g1, g2, coil_width_inner_ratio, coil_width_outer_ratio, coil_width_x_ratio, coil_height, space_height,
    # space_inner_ratio, space_outer_ratio, space_x_ratio, move_z, ratio]
    
    def _evaluate(self, X, out, *args, **kwargs) :

        copper_data = pre_processing_data_copper(X)

        tx_val = 4
        rx_val = 11
        n = copper_data.shape[0]

        # 스칼라 → (n,1) 컬럼으로 확장
        tx_col = np.full((n, 1), tx_val, dtype=copper_data.dtype)
        rx_col = np.full((n, 1), rx_val, dtype=copper_data.dtype)

        # 열 붙이기
        copper_data = np.hstack((copper_data, tx_col, rx_col))


        
        copperloss_Tx = models["copperloss_Tx"].predict(copper_data) 
        copperloss_Rx1 = models["copperloss_Rx1"].predict(copper_data)
        copperloss_Rx2 = models["copperloss_Rx2"].predict(copper_data)
        
        Lmt = models["Lmt"].predict(copper_data)
        Llt = models["Llt"].predict(copper_data)

        freq_one = X[:,0]
        freq = freq_one * 5

        input_voltage_one = X[:,1]
        input_voltage = input_voltage_one * 50 + 100


        magnetizing_current = input_voltage*math.sqrt(2)/2/math.pi/freq/10**3/Lmt/10**(-6) # 제일 마지막 /2 시뮬 돌릴때 잘못넣음 시뮬은 수정해서 다음번에 NSGA 복사할때 /2 제거해야함

        core_data = pre_processing_data_core(X,tx_col,magnetizing_current)

        coreloss = models["coreloss"].predict(core_data)
        magnetizing_copperloss_Tx = models["magnetizing_copperloss_Tx"].predict(core_data)
        magnetizing_copperloss_Rx1 = models["magnetizing_copperloss_Rx1"].predict(core_data)
        magnetizing_copperloss_Rx2 = models["magnetizing_copperloss_Rx2"].predict(core_data)


        Area, height, max_length, Tx_Rx, hh, gap, Volume = compute_area(X)
        # Core volume
 

        #Total loss
        total_loss = copperloss_Tx + copperloss_Rx1 + copperloss_Rx2 + coreloss + magnetizing_copperloss_Tx + magnetizing_copperloss_Rx1 + magnetizing_copperloss_Rx2

        #calculate B field

        B_center = models["B_center"].predict(core_data)
        B_left = models["B_left"].predict(core_data)
        B_top_left = models["B_top_left"].predict(core_data)



        gLmt = (Lmt - 37)*(Lmt - 43)
        gh = (height - 4.5)*(height-5)

        gB_center =  (B_center-0.3)
        gB_left =  (B_left-0.3)
        gB_top_left =  (B_top_left-0.3)


        gmax_length = -(max_length-0.00001)
        ghh = -(hh - 0.00001)
        ggap = -(gap - 0.00001)
        gTx_Rx = -(Tx_Rx - 0.0004)
        ginput_voltage = (input_voltage - 380)*(input_voltage - 420)
        gfreq = (freq - 100)*(freq - 150)

        # print(f"Tx:{Tx}")
        # print(f"B_left:{B_left}")
        # print(f"B_top_left:{B_top_left}")
    
        



        out["F"] = np.column_stack([Volume,total_loss]) # "Minimize" values (volume, coreloss)
        #out["G"] = np.column_stack([gB, gxgap, gygap1, gygap2, gAT, gAR, gBT, gBR])
        out["G"] = np.column_stack([gLmt,gh,gB_center,ghh,gmax_length,ggap,gB_left,gB_top_left,gTx_Rx,ginput_voltage,gfreq])
        out["magnetizing_current"]   = magnetizing_current
        out["input_voltage"]          = input_voltage
        out["copper_data"]          = copper_data 
        out["core_data"]           = core_data
        out["Area"]              = Area
        out["Volume"]            = Volume

my_vars = ProblemVariables(vars1)

def run_nsga2(seed, problem):
    np.random.seed(seed)
    random.seed(seed)

    algorithm = NSGA2(
        pop_size=MU,
        sampling=IntegerRandomSampling(),
        crossover=SBX(prob=CXPB, eta=15, repair=RoundingRepair()),
        mutation=PM(eta=20, repair=RoundingRepair()),
        eliminate_duplicates=True
    )

    return minimize(
        problem= problem,
        algorithm=algorithm,
        termination=('n_gen', NGEN),
        seed=seed,
        verbose=True
    )

MU, NGEN = 100, 400
CXPB, MUTPB = 0.7, 0.3
NUM_ITRS = 5
feature_names = ["freq","w1","l1_leg","l1_top","l2","h1","l1_center","Tx_turns","Tx_width",
                 "Tx_height","Tx_space_x","Tx_space_y","Tx_preg",
                 "Rx_width","Rx_height","Rx_space_x","Rx_space_y","Rx_preg",
                 "g2","Tx_layer_space_x","Tx_layer_space_y",
                 "Tx_current","Rx_current"] 
                
def build_df_from_result(res):
    """pymoo 결과(res) → 파레토 해 집합 DataFrame (벡터화 버전)"""
    # 1) Pareto set (X) 전체를 2-D 로 확보

    # 2) pre-processing (벡터화)
    magnetizing_current = res.pop.get("magnetizing_current")
    input_voltage    = res.pop.get("input_voltage")
    copper_data      = res.pop.get("copper_data")          # shape (n, p)    # shape (n, p+1)
    core_data      = res.pop.get("core_data")
    X_design      = res.X.astype(int)                # 원본 정수 설계벡터
    n             = len(res.pop)
    Area          = res.pop.get("Area")
    Volume        = res.pop.get("Volume")



    # 3) ML 모델 일괄 예측
    copperloss_Tx = models["copperloss_Tx"].predict(copper_data)      # (n,)
    copperloss_Rx1 = models["copperloss_Rx1"].predict(copper_data)
    copperloss_Rx2 = models["copperloss_Rx2"].predict(copper_data)
    Lmt          = models["Lmt"].predict(copper_data)
    Llt          = models["Llt"].predict(copper_data)

    magnetizing_copperloss_Tx = models["magnetizing_copperloss_Tx"].predict(core_data)
    magnetizing_copperloss_Rx1 = models["magnetizing_copperloss_Rx1"].predict(core_data)
    magnetizing_copperloss_Rx2 = models["magnetizing_copperloss_Rx2"].predict(core_data)

    coreloss      = models["coreloss"].predict(core_data)
    B_center      = models["B_center"].predict(core_data)
    B_top_left    = models["B_top_left"].predict(core_data)
    B_left        = models["B_left"].predict(core_data)


    total_loss    = copperloss_Tx + copperloss_Rx1 + copperloss_Rx2 + coreloss + magnetizing_copperloss_Tx + magnetizing_copperloss_Rx1 + magnetizing_copperloss_Rx2

    
    # 5) DataFrame assemble
    df = pd.DataFrame(copper_data, columns=feature_names)
    df = df.assign(
        magnetizing_current = magnetizing_current,
        input_voltage       = input_voltage,
        Lmt                 = Lmt,
        Llt                 = Llt,
        copperloss_Tx       = copperloss_Tx,
        copperloss_Rx1       = copperloss_Rx1,
        copperloss_Rx2       = copperloss_Rx2,
        coreloss            = coreloss,
        magnetizing_copperloss_Tx = magnetizing_copperloss_Tx,
        magnetizing_copperloss_Rx1 = magnetizing_copperloss_Rx1,
        magnetizing_copperloss_Rx2 = magnetizing_copperloss_Rx2,
        total_loss          = total_loss,
        B_center            = B_center,
        B_left              = B_left,
        Area                = Area,
        Volume              = Volume
        # 필요 시
    )

    return df
    
pareto_file = 'pareto_front.csv'
loop_counter_file = 'loop_counter.txt'
best_values_file = 'best_values.csv'

import time
problem_instance = MyProblem(my_vars)
all_results = []

pareto_file = 'pareto_front.csv'
loop_counter_file = 'loop_counter.txt'
best_values_file = 'best_values.csv'

for itr in range(NUM_ITRS):
    print(f"Running NSGA-II {itr+1} / {NUM_ITRS}")
    seed = np.random.randint(0, 1000000)
    
    try:
        res = run_nsga2(seed, problem_instance)
        df = build_df_from_result(res)

        if df is not None:
            print(f"NSGA-II 완료: {len(df)} 개의 해 발견")

            
            # 2. 파일 락 획득 및 읽기
            print("파일 락 획득 시도...")
            while True:
                try:
                    with open(pareto_file, 'a+') as f_pareto, \
                         open(best_values_file, 'a+') as f_best, \
                         open(loop_counter_file, 'a+') as f_counter:
                        

                        # 파일 읽기
                        f_pareto.seek(0)
                        f_best.seek(0)
                        f_counter.seek(0)
                        
                        current_pareto = pd.read_csv(f_pareto) if os.path.getsize(pareto_file) > 0 else pd.DataFrame()
                        current_best_values = pd.read_csv(f_best) if os.path.getsize(best_values_file) > 0 else pd.DataFrame(columns=['iteration', 'max_Volume', 'min_loss'])
                        current_loop_counter = int(f_counter.read().strip()) if os.path.getsize(loop_counter_file) > 0 else 0
                        
                        print(f"현재 파레토 크기: {len(current_pareto)} 행")
                        print(f"현재 best_values 크기: {len(current_best_values)} 행")
                        print(f"현재 루프 카운터: {current_loop_counter}")
                        
                        # 3. 데이터 처리 (빠른 처리)
                        print("데이터 처리 시작...")
                        df_current = pd.concat([df, current_pareto], ignore_index=True)
                        print(len(df_current))
                        F = df_current[["Volume", "total_loss"]].to_numpy()
                        
                        nds = NonDominatedSorting().do(F, only_non_dominated_front=True)
                        df_pareto = df_current.iloc[nds].copy()
                        df_pareto["total_loss"] = F[nds, 1]  # 효율 부호 복원
                        df_pareto = df_pareto.sort_values(by="total_loss", ascending=False).reset_index(drop=True)
                        
                        max_loss = df['total_loss'].min()
                        min_Volume = df['Volume'].max()
                        
                        new_loop_counter = current_loop_counter + 1
                        
                        new_row = pd.DataFrame({
                            'iteration': [new_loop_counter],
                            'max_loss': [max_loss],
                            'min_Volume': [min_Volume]
                        })
                        
                        updated_best_values = pd.concat([current_best_values, new_row], ignore_index=True)
                        
                        print(f"논 도미네이티드 소팅 완료: {len(df_pareto)} 행")
                        
                        # 4. 파일 쓰기
                        print("파일 쓰기 시작...")
                        f_pareto.seek(0)
                        f_pareto.truncate()
                        df_pareto.to_csv(f_pareto, index=False)
                        
                        f_best.seek(0)
                        f_best.truncate()
                        updated_best_values.to_csv(f_best, index=False)
                        
                        f_counter.seek(0)
                        f_counter.truncate()
                        f_counter.write(str(new_loop_counter))
                        
                        # 백업 생성 (10회마다)
                        # if new_loop_counter % 10 == 0 or new_loop_counter == 1:
                        if True :
                            backup_file = f'pareto_front_backup_{new_loop_counter}.csv'
                            df_pareto.to_csv(backup_file, index=False)
                            print(f"Backup created: {backup_file}")
                        
                        print(f"Iteration {itr+1} 완료 (Total loops: {new_loop_counter})")
                        print(f"Max loss: {max_loss:.4f}, Min Volume: {min_Volume:.4f}")
                        print("=" * 50)
                        
                        # 락 해제는 with 블록을 나가면서 자동으로 수행됨
                        break
                except BlockingIOError:
                    print("다른 프로세스가 파일을 사용 중입니다. 1초 후 재시도...")
                    time.sleep(1)
                    continue
            
    except Exception as e:
        print(f"Error in iteration {itr+1}: {str(e)}")
        continue

# 최종 결과 출력
if 'df_pareto' in locals():
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    print("Final Pareto front:")
    print(df_pareto)
else:
    print("No valid results found")


Running NSGA-II 1 / 5
Error in iteration 1: 'NoneType' object is not callable
Running NSGA-II 2 / 5
Error in iteration 2: 'NoneType' object is not callable
Running NSGA-II 3 / 5
Error in iteration 3: 'NoneType' object is not callable
Running NSGA-II 4 / 5
Error in iteration 4: 'NoneType' object is not callable
Running NSGA-II 5 / 5
Error in iteration 5: 'NoneType' object is not callable
No valid results found


In [5]:
# problem_instance = MyProblem(my_vars)
# run_nsga2(seed, problem_instance)

In [6]:
import matplotlib.pyplot as plt

# pymoo 결과 res에서 F를 꺼내 옵니다 (shape = [n_points, 2])
F = df_pareto
# F = final_F

# 1열이 f1, 2열이 f2

plt.figure()

f1_vals = ((F["l1_leg"] + F["l2"] + F["l1_center"]/2)*2) * (F["w1"] + (F["Tx_space_x"]+(F["Tx_width"]+F["Tx_layer_space_x"])*F["Tx_turns"]+F["Rx_space_x"]+F["Rx_width"])) * (F["h1"] + F["l1_top"]*2) * 1e-3
f2_vals = F["total_loss"]
plt.scatter(f1_vals, f2_vals)
plt.xlabel('Volume [cm³]') 
plt.ylabel('total loss [W]')
plt.title('EVDD transformer design')
plt.grid(True)
plt.show()

NameError: name 'df_pareto' is not defined

In [ ]:
df_pareto

,freq,w1,l1_leg,l1_top,l2,h1,l1_center,Tx_turns,Tx_width,Tx_height,Tx_space_x,Tx_space_y,Tx_preg,Rx_width,Rx_height,Rx_space_x,Rx_space_y,Rx_preg,g2,Tx_layer_space_x,Tx_layer_space_y,wire_diameter,strand_number,Tx_current,Rx_current,magnetizing_current,input_voltage,Lmt,Llt,copperloss_Tx,copperloss_Rx1,copperloss_Rx2,coreloss,magnetizing_copperloss_Tx,magnetizing_copperloss_Rx1,magnetizing_copperloss_Rx2,total_loss,B_center,B_left,Area,Volume
0,282.0,20.0,2.0,1.2,6.9,2.10,4.0,16.0,0.187083,0.187083,0.1,0.6,0.05,4.0,0.1,0.3,0.1,0.08,0.66,0.2,0.3,0.05,7.0,1.0,1.7,0.656476,100.0,42.985540,25.246256,0.684761,0.036096,0.013863,0.067029,0.732599,0.077726,0.313171,1.925244,0.057256,0.123457,569.752033,2563.884147
1,284.0,20.0,2.0,1.2,7.0,2.10,4.0,18.0,0.187083,0.187083,0.1,0.7,0.05,4.0,0.1,0.4,0.1,0.09,0.66,0.2,0.3,0.05,7.0,1.0,1.7,0.652209,100.0,42.962037,24.971516,0.723536,0.035407,0.013694,0.046847,0.476931,0.079381,0.308430,1.684226,0.056111,0.116548,577.179116,2597.306020
2,285.0,20.0,2.0,1.2,7.1,2.10,4.0,16.0,0.187083,0.187083,0.1,0.8,0.05,4.0,0.1,0.4,0.1,0.09,0.67,0.2,0.3,0.05,7.0,1.1,1.5,0.694299,105.0,42.226803,24.699917,0.722793,0.024350,0.011879,0.039118,0.464147,0.079126,0.228268,1.569679,0.056263,0.116326,582.426198,2620.917893
3,285.0,20.0,2.0,1.2,7.2,2.10,4.0,17.0,0.187083,0.187083,0.1,0.8,0.05,4.1,0.1,0.2,0.1,0.10,0.69,0.2,0.3,0.05,7.0,1.1,1.1,0.662513,100.0,42.145469,24.490215,0.745928,0.018626,0.011130,0.042506,0.453478,0.074900,0.217621,1.564191,0.052977,0.119134,585.433281,2634.449766
4,282.0,20.0,2.0,1.2,7.2,2.10,4.0,17.0,0.187083,0.187083,0.1,0.8,0.05,4.1,0.1,0.3,0.1,0.11,0.67,0.2,0.3,0.05,7.0,1.0,1.3,0.685222,103.0,42.417658,24.440622,0.708460,0.022333,0.012294,0.034601,0.465172,0.076926,0.229472,1.549257,0.055156,0.118952,587.673281,2644.529766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,150.0,20.0,13.6,1.3,21.3,2.00,21.0,9.0,0.456946,0.456946,0.1,1.8,0.21,13.1,0.1,1.0,0.3,0.29,0.99,0.2,0.9,0.06,29.0,1.1,1.4,1.431868,102.0,37.791658,14.116882,0.121362,0.014286,0.008213,0.006023,0.020635,0.009731,0.009490,0.189740,0.075036,0.025002,3385.453659,15573.086833
136,151.0,20.0,13.6,1.3,21.5,2.01,21.0,9.0,0.456946,0.456946,0.1,1.8,0.21,13.1,0.1,1.1,0.3,0.29,0.99,0.2,0.9,0.06,29.0,1.1,1.4,1.421493,102.0,37.815407,14.054333,0.107119,0.013592,0.008031,0.005209,0.018551,0.012423,0.009602,0.174526,0.074674,0.024979,3409.487552,15717.737616
137,150.0,20.0,13.6,1.3,21.4,2.01,21.0,9.0,0.456946,0.456946,0.1,1.8,0.21,13.1,0.1,1.0,0.3,0.29,0.99,0.3,0.9,0.06,29.0,1.0,1.4,1.431912,102.0,37.790520,14.118030,0.107012,0.013681,0.008013,0.006130,0.022664,0.006024,0.009998,0.173521,0.082970,0.025148,3429.310606,15809.121893
138,151.0,20.0,13.6,1.3,21.3,2.01,21.0,9.0,0.456946,0.456946,0.1,1.8,0.21,13.1,0.1,1.1,0.3,0.29,0.99,0.3,0.9,0.06,29.0,1.1,1.4,1.421461,102.0,37.816243,14.163163,0.106982,0.013684,0.007976,0.005134,0.021871,0.007959,0.009719,0.173325,0.083584,0.025220,3430.853659,15816.235370
